In [7]:
import os
from dotenv import load_dotenv, get_key
load_dotenv()

True

In [5]:
from langchain.agents import Tool
from langchain_community.tools.semanticscholar.tool import SemanticScholarAPIWrapper

semantic_scholar = SemanticScholarAPIWrapper()

tools = []

tools.append(
   Tool(
       name="Semantic Scholar Search",
       func=semantic_scholar.run,
       description="Useful for retrieving academic references, citations, and publications.",
   )
)

print(tools)


[Tool(name='Semantic Scholar Search', description='Useful for retrieving academic references, citations, and publications.', func=<bound method SemanticScholarAPIWrapper.run of SemanticScholarAPIWrapper(semanticscholar_search=<bound method SemanticScholar.search_paper of <semanticscholar.SemanticScholar.SemanticScholar object at 0x11465d5d0>>, top_k_results=5, S2_MAX_QUERY_LENGTH=300, load_max_docs=100, doc_content_chars_max=4000, returned_fields=['title', 'abstract', 'venue', 'year', 'paperId', 'citationCount', 'openAccessPdf', 'authors', 'externalIds'])>)]


In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import StructuredTool

def create_reference(publications: str) -> str:
    """Prepare a list of IEEE bibliograpy formatter reference from given list of publications."""

    prompt_extract = (
        "You are an assistant tasked with creating IEEE styled referece for citation from provided publications. "
        "Now, please generate IEEE bibliography styled references for following publications.\n\n"
        "Publications: {publications}"
    )

    prompt = PromptTemplate.from_template(prompt_extract)
    chain = prompt | llm
    processed_chunks = []
    for chunk in chunks:
        result = chain.invoke({"publications": chunk})
        processed_chunks.append(result.content)

    return "\n".join(processed_chunks)

reference_tool = Tool(
    name="Reference Creator",
    description="Create IEEE conventional reference for citation from a list of academic publications.",
    func=create_reference
)

tools.append(reference_tool)

print(tools)

[Tool(name='Semantic Scholar Search', description='Useful for retrieving academic references, citations, and publications.', func=<bound method SemanticScholarAPIWrapper.run of SemanticScholarAPIWrapper(semanticscholar_search=<bound method SemanticScholar.search_paper of <semanticscholar.SemanticScholar.SemanticScholar object at 0x11465d5d0>>, top_k_results=5, S2_MAX_QUERY_LENGTH=300, load_max_docs=100, doc_content_chars_max=4000, returned_fields=['title', 'abstract', 'venue', 'year', 'paperId', 'citationCount', 'openAccessPdf', 'authors', 'externalIds'])>), Tool(name='Reference Creator', description='Create IEEE conventional reference for citation from a list of academic publications.', func=<function create_reference at 0x124326fc0>)]


In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=os.getenv('GEMINI_API_KEY'),
    temperature=0.1
)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    max_iterations=3,
    handle_parsing_errors=True,
    verbose=True,
)

In [26]:
chat_history = []
user_input = "Can you find top three most relevant publications about Beamforming, point-to-point multiple-input multiple-output?"
response = agent.run(input=user_input, chat_history=chat_history)
chat_history.append(("User: " + user_input, "Agent: " + response))

print("Final Content for Query:", response)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Semantic Scholar Search
Action Input: Beamforming, point-to-point multiple-input multiple-output
Observation: Published year: 2024
Title: Joint Power Allocation and Hybrid Beamforming for Cell-Free mmWave Multiple-Input Multiple-Output with Statistical Channel State Information
Authors: Jiawei Bai, Guangying Wang, Ming Wang, Jinjin Zhu
Abstract: Cell-free millimeter wave (mmWave) multiple-input multiple-output (MIMO) can effectively overcome the shadow fading effect and provide macro gain to boost the throughput of communication networks. Nevertheless, the majority of the existing studies have overlooked the user-centric characteristics and practical fronthaul capacity limitations. To solve these practical problems, we introduce a resource allocation scheme using statistical channel state information (CSI) for uplink user-centric cell-free mmWave MIMO system. The hybrid beamforming (HBF) architecture 